In [1]:
import pickle
import numpy as np
import pandas as pd
import os
import csv
from scipy import stats
from order_fulfillment_environment_notidentical_arrival_probs import OrderFulfillment
from cluster_fulfillment_policy_notidentical_arrival_probs import FulfillmentPolicy
from fulfillment_policy_Will_Ma import WillMaFulfillmentPolicy

In [2]:
num_items = 20
n_0 = 5
p_stock = 0.75
CSL=0.5

conservative_prob_sequence=[0, 0.01, 0.05, 0.1, 0.15, 0.2, 1]

num_order_sequences = 100

facility_path = "fulfillment_centers_warmup.csv"
cities_path = "cities_warmup.csv"
home_path = "Data/"
facilities_path = os.path.join(home_path, facility_path)
cities_path =  os.path.join(home_path, cities_path)

In [3]:
def DLP_ours_results(fulfillment_policy, print_optimal_values=False):
    # Call the method and get the results
    results = fulfillment_policy.get_optimization_results

    # Check if the optimization was successful and optionally print the results
    if results is not None:
        if print_optimal_values:
            print("Optimal 'x' values:")
            for q, x_values in results["x_values"].items():
                print(f"Order Type {q}:")
                for method_index, value in x_values.items():
                    print(f"  Method {method_index}: {value}")

            print(f"\nNumber of decision variables: {results['num_vars']}")
            print(f"Number of constraints: {results['num_constrs']}")
            print(f"Optimal objective value: {results['optimal_value']}")

        return results["num_vars"], results["num_constrs"], results["optimal_value"]
    
    else:
        print("Optimization did not find an optimal solution for our formulation.")
        return None

In [4]:
def evaluate_policies(n_max, T, alpha, instance, num_order_sequences=num_order_sequences, num_items=num_items, n_0=n_0, p_stock=p_stock, conservative_prob_sequence=conservative_prob_sequence):
    
    dir_results = f'/Users/ayoubamil/Documents/GitHub/multi-item-order-fulfillment_AA_AM_YW/LP_results_instance={instance}/n_max={n_max}/T={T}_alpha={alpha}'

    # Load LP results from files (make sure to run cluster_solve_LP.py first with same order_fulfillment parameters)
    with open(os.path.join(dir_results, 'LP_solution.pkl'), 'rb') as f:
        LP_solution = pickle.load(f)
    with open(os.path.join(dir_results, 'methods.pkl'), 'rb') as f:
        methods = pickle.load(f)
    with open(os.path.join(dir_results, 'sizes.pkl'), 'rb') as f:
        sizes = pickle.load(f)
    with open(os.path.join(dir_results, 'consumption_probability_lists.pkl'), 'rb') as f:
        consumption_probability_lists = pickle.load(f)
    with open(os.path.join(dir_results, 'get_optimization_results.pkl'), 'rb') as f:
        get_optimization_results = pickle.load(f)
    
    dir_results_wm = f'/Users/ayoubamil/Documents/GitHub/multi-item-order-fulfillment_AA_AM_YW/LP_results_WM_instance={instance}/n_max={n_max}/T={T}_alpha={alpha}'
        
    with open(os.path.join(dir_results_wm, 'wm_num_vars.pkl'), 'rb') as f:
        wm_num_vars = pickle.load(f)
    with open(os.path.join(dir_results_wm, 'wm_num_constrs.pkl'), 'rb') as f:
        wm_num_constrs = pickle.load(f)
    with open(os.path.join(dir_results_wm, 'wm_optimal_value.pkl'), 'rb') as f:
        wm_optimal_value = pickle.load(f)
    with open(os.path.join(dir_results_wm, 'optimal_u.pkl'), 'rb') as f:
        optimal_u = pickle.load(f)
    with open(os.path.join(dir_results_wm, 'optimal_y.pkl'), 'rb') as f:
        optimal_y = pickle.load(f)
    with open(os.path.join(dir_results_wm, 'wm_optimization_duration.pkl'), 'rb') as f:
        wm_optimization_duration = pickle.load(f)
    with open(os.path.join(dir_results_wm, 'our_optimization_duration.pkl'), 'rb') as f:
        our_optimization_duration = pickle.load(f)
    
    # This object has the be the same as the one used to generate the LP results in the cluster_solve_LP.py    
    order_fulfillment = OrderFulfillment(num_items=num_items, n_max=n_max, n_0=n_0, p_stock=p_stock, T=T, CSL=CSL, 
                                        facilities_data=facilities_path, 
                                        cities_data=cities_path, 
                                        prob_seed_value=instance, 
                                        order_seed_value=instance, 
                                        inv_seed_value=instance, 
                                        alpha=alpha)

    # Now create an instance of FulfillmentPolicy with these parameters
    fulfillment_policy = FulfillmentPolicy(order_fulfillment, LP_solution, methods, sizes, consumption_probability_lists, get_optimization_results)
    
    our_num_vars, our_num_constrs, our_optimal_value = DLP_ours_results(fulfillment_policy)
    
    # Instantiate WillMaFulfillmentPolicy
    will_ma_policy = WillMaFulfillmentPolicy(order_fulfillment, optimal_u)
    
    results_over_conservative_probs = {}
    
    for conservative_prob in conservative_prob_sequence:
        # Generate magician problems (dictionary where the keys are (i,k))
        cdfs_magician_problems = fulfillment_policy.solve_cdfs_magician_problems(conservative_prob=conservative_prob)
        
        # FOR EACH INSTANCE, GENERATE DIFFERENT ORDER SEQUENCES (i.e. order arrivals through time)
        order_sequences = np.arange(1, num_order_sequences + 1) # seed_value for each order sequence

        # Initialize counters for number of times each policy is better
        count_fulfillment_policy = 0
        count_will_ma_policy = 0
        tie = 0

        # Lists to store costs
        fulfillment_costs_list = []
        always_accept_costs_list = []
        will_ma_costs_list = []
        
        # Lists to store sampled orders, samples methods, and accepts decisions
        our_policy_orders_methods_decisions = {}
        aa_policy_orders_methods_decisions = {}
        
        for order_sequence in order_sequences:
            
            magician_problems = fulfillment_policy.solve_magician_problems(conservative_prob=conservative_prob, cdfs_magician_problems=cdfs_magician_problems)
            
            # Initialize inventory consumption for our fulfillment policy
            inventory_consumption = fulfillment_policy.initialize_inventory_consumption()
            sampled_orders_index, sampled_orders, sampled_methods, accepts_decisions, fulfillment_costs = fulfillment_policy.new_fulfillment_policy_end_modified(inventory_consumption, magician_problems, seed_value=order_sequence)
            fulfillment_policy.check_consistency(inventory_consumption)
            total_fulfillment_cost = sum(fulfillment_costs)
            our_policy_orders_methods_decisions["orders"] = sampled_orders
            our_policy_orders_methods_decisions["methods"] = sampled_methods
            our_policy_orders_methods_decisions["accepts"] = accepts_decisions
        
            # Initialize inventory consumption for always_accept_policy
            inventory_consumption_aa = fulfillment_policy.initialize_inventory_consumption()
            sampled_orders_index_aa, sampled_orders_aa, sampled_methods_aa, accepts_decisions_aa, fulfillment_costs_aa = fulfillment_policy.always_accept_policy(inventory_consumption_aa, seed_value=order_sequence)
            fulfillment_policy.check_consistency(inventory_consumption)
            total_always_accept_cost = sum(fulfillment_costs_aa)
            aa_policy_orders_methods_decisions["orders"] = sampled_orders_aa
            aa_policy_orders_methods_decisions["methods"] = sampled_methods_aa
            aa_policy_orders_methods_decisions["accepts"] = accepts_decisions_aa

            # Will Ma's policy
            will_ma_cost = will_ma_policy.run(sampled_orders, seed_value=order_sequence)

            # Append cost of order sequence to lists
            fulfillment_costs_list.append(total_fulfillment_cost)
            always_accept_costs_list.append(total_always_accept_cost)
            will_ma_costs_list.append(will_ma_cost)

            # Check which policy has lower costs (for given order arrivals) and update counters accordingly
            if total_fulfillment_cost < will_ma_cost:
                count_fulfillment_policy += 1
            elif will_ma_cost < total_fulfillment_cost:
                count_will_ma_policy += 1
            elif total_fulfillment_cost == will_ma_cost:
                tie += 1

        # 95% CONFIDENCE INTERVAL for the difference (for fixed instance) between our policy and Will Ma's policy
        cost_differences = np.array(fulfillment_costs_list) - np.array(will_ma_costs_list)
        mean_difference = np.mean(cost_differences)
        std_deviation = np.std(cost_differences, ddof=1)
        n = len(cost_differences)
        standard_error = std_deviation / np.sqrt(n) # Calculate the standard error and the t-score for 95% confidence
        t_score = stats.t.ppf(0.975, df=n-1)  # two-tailed 95% confidence, so 0.975
        confidence_interval = (mean_difference - t_score * standard_error, mean_difference + t_score * standard_error)
        
        # 95% CONFIDENCE INTERVAL for the difference between our policy and AA policy
        cost_differences_aa = np.array(fulfillment_costs_list) - np.array(always_accept_costs_list)
        mean_difference_aa = np.mean(cost_differences_aa)
        std_deviation_aa = np.std(cost_differences_aa, ddof=1)
        n_aa = len(cost_differences_aa)
        standard_error_aa = std_deviation_aa / np.sqrt(n_aa)  # Calculate the standard error
        t_score_aa = stats.t.ppf(0.975, df=n_aa-1)  # two-tailed 95% confidence, so 0.975
        confidence_interval_aa = (mean_difference_aa - t_score_aa * standard_error_aa, mean_difference_aa + t_score_aa * standard_error_aa)
        
        expected_policy_cost = round(np.mean(fulfillment_costs_list),2) # expected cost over the number of order sequences
        expected_policy_cost_aa = round(np.mean(always_accept_costs_list),2) # expected cost over the number of order sequences
        expected_cost_will_ma = round(np.mean(will_ma_costs_list), 2)
        expected_cost_difference = round(expected_policy_cost - expected_cost_will_ma,2)
        expected_cost_difference_aa = round(expected_policy_cost - expected_policy_cost_aa,2)  # expected cost difference between our policy and AA policy
        
        percent_ours_better_over_order_arrivals = count_fulfillment_policy / num_order_sequences * 100
        percent_equal_over_order_arrivals = tie / num_order_sequences * 100
        percent_wm_better_over_order_arrivals = count_will_ma_policy / num_order_sequences * 100
    
        results_per_conservative_probs = {
            "fulfillments_cost_our_policy_per_order_sequence": fulfillment_costs_list,
            "fulfillments_cost_aa_per_order_sequence": always_accept_costs_list,
            "fulfillments_cost_will_ma_per_order_sequence": will_ma_costs_list,
            "expected_cost_our_policy_over_order_sequences": expected_policy_cost,
            "expected_cost_aa_over_order_sequence": expected_policy_cost_aa,
            "expected_cost_will_ma_over_order_sequences": expected_cost_will_ma,
            "expected_cost_difference_over_order_sequence_our_cost-wm_cost": expected_cost_difference,
            "95%_CI_cost_difference_our_cost-wm_cost": confidence_interval,
            "expected_cost_difference_over_order_sequence_our_cost-aa_cost": expected_cost_difference_aa,
            "95%_CI_cost_difference_our_cost-aa_cost": confidence_interval_aa,
            "percent_ours_better_over_order_arrivals": percent_ours_better_over_order_arrivals,
            "percent_equal_over_order_arrivals": percent_equal_over_order_arrivals,
            "percent_wm_better_over_order_arrivals": percent_wm_better_over_order_arrivals,
            "(our_num_vars, our_num_constrs)": (our_num_vars, our_num_constrs),
            "our_optimal_value": our_optimal_value,
            "our_optimization_duration": our_optimization_duration,
            "(wm_num_vars, wm_num_constrs)": (wm_num_vars, wm_num_constrs),
            "wm_optimal_value": wm_optimal_value,
            "wm_optimization_duration": wm_optimization_duration,
        }
    
        results_over_conservative_probs[f"{conservative_prob}"] = results_per_conservative_probs
    
    res_path = f"Results/T={T}/num_items={num_items}_n_max={n_max}_n_0={n_0}/instance={instance}_T={T}_alpha={alpha}"
    
    if not os.path.exists(res_path):
        os.makedirs(res_path)

    # Define the CSV file path
    csv_file_path = os.path.join(res_path, f'instance={instance}_T={T}_alpha={alpha}.csv')

    # Prepare data for CSV
    data_for_csv = []
    for prob, results in results_over_conservative_probs.items():
        for key, value in results.items():
            if isinstance(value, list):
                for idx, val in enumerate(value):
                    data_row = {
                        "Conservative Probability": prob,
                        "Metric": key,
                        "Order Sequence": idx + 1,
                        "Value": val
                    }
                    data_for_csv.append(data_row)
            elif isinstance(value, tuple):
                data_row = {
                    "Conservative Probability": prob,
                    "Metric": key,
                    "Order Sequence": "N/A",
                    "Value": f"({value[0]:.2f}, {value[1]:.2f})"
                }
                data_for_csv.append(data_row)
            else:
                data_row = {
                    "Conservative Probability": prob,
                    "Metric": key,
                    "Order Sequence": "N/A",
                    "Value": value
                }
                data_for_csv.append(data_row)

    # Write to CSV
    with open(csv_file_path, 'w', newline='') as csvfile:
        fieldnames = ["Conservative Probability", "Metric", "Order Sequence", "Value"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for row in data_for_csv:
            writer.writerow(row)

In [5]:
T = 100
n_max_values = [5]
alpha_values = [0, 0.5, 1]

num_instances = 30

for instance in range(1, num_instances+1):
    for n_max in n_max_values:
            for alpha in alpha_values:
                evaluate_policies(n_max, T, alpha, instance)

expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the available ones
expected number of broken wands larger than the availab

In [6]:
# # Initialize dictionary to store aggregated results
# aggregated_results = {}

# T = 10
# instances = 30
# alpha_values = [0, 0.5, 0.75, 0.9, 1]
# n_max_values = [2, 5]
# conservative_prob_sequence = [0, 0.01, 0.1, 0.2, 0.25, 0.3, 0.5, 0.7, 1]

# # Iterate over alpha and n_max combinations
# for alpha in alpha_values:
#     for n_max in n_max_values:
#         # Initialize data structure for storing results for this alpha, n_max combination across all instances
#         key = (alpha, n_max)
#         aggregated_results[key] = {str(prob): None for prob in conservative_prob_sequence}

#         # Accumulate and average results for each conservative probability across instances
#         for instance in range(1, instances + 1):
#             instance_results = evaluate_policies(n_max, T, alpha, instance, 
#                                                  num_order_sequences=num_order_sequences, num_items=num_items, 
#                                                  n_0=n_0, p_stock=p_stock, 
#                                                  conservative_prob_sequence=conservative_prob_sequence)
#             # Aggregate instance results
#             for prob in conservative_prob_sequence:
#                 prob_str = str(prob)
#                 if aggregated_results[key][prob_str] is None:
#                     aggregated_results[key][prob_str] = {metric: 0 for metric in instance_results[prob_str]}
                
#                 for metric in instance_results[prob_str]:
#                     aggregated_results[key][prob_str][metric] += instance_results[prob_str][metric]       

In [7]:
# for alpha in alpha_values:
#     for n_max in n_max_values:
#         # Compute averages over instances
#         for prob in conservative_prob_sequence:
#             prob_str = str(prob)
#             for metric in aggregated_results[key][prob_str]:
#                 aggregated_results[key][prob_str][metric] /= instances

# # Print or store the averaged results
# for (alpha, n_max), probs in aggregated_results.items():
#     print(f"Alpha: {alpha}, n_max: {n_max}")
#     for prob, metrics in probs.items():
#         print(f"Conservative Probability: {prob}")
#         for metric, value in metrics.items():
#             print(f"  {metric}: {value}")
#     print()

In [8]:
# T = 10
# instance = 1
# alpha = 0.5
# n_max = 2

# path = f'Results/num_items=20_n_max={n_max}_n_0=5/instance={instance}_T={T}_alpha={alpha}/instance={instance}_T={T}_alpha={alpha}.csv'
# df = pd.read_csv(path)

# pd.set_option('display.max_rows', None)

# df

In [9]:
T = 100
alpha = 0
n_max = 2

instances = np.arange(1, 50 + 1)
nu_values = [0, 0.01, 0.1, 0.2, 0.25, 0.3, 0.5, 0.7, 1]

results = {}

competitive_ratio = {}

for nu in nu_values:
    
    results[nu] = {}
    
    competitive_ratio[nu] = {}
    
    expected_cost_our_policy = []
    expected_cost_aa = []
    expected_cost_will_ma = []

    expected_cost_difference_our_policy_wm = []
    CI_cost_difference_our_cost_wm_cost = []

    expected_cost_difference_our_policy_aa = []
    CI_cost_difference_our_cost_aa_cost = []
    
    our_number_of_variables_constraints_LP = []
    our_optimal_value_LP = []
    our_runtime_LP = []
    wm_number_of_variables_constraints_LP = []
    wm_optimal_value_LP = []
    wm_runtime_LP = []
    
    for instance in instances:
        path = f'Results/num_items=20_n_max={n_max}_n_0=5/instance={instance}_T={T}_alpha={alpha}/instance={instance}_T={T}_alpha={alpha}.csv'
        df = pd.read_csv(path)
        
        if nu == 0:
            expected_cost_our_policy.append(float(df.iloc[90][3]))
            expected_cost_aa.append(float(df.iloc[91][3]))
            expected_cost_will_ma.append(float(df.iloc[92][3]))
            
            expected_cost_difference_our_policy_wm.append(float(df.iloc[93][3]))
            CI_cost_difference_our_cost_wm_cost.append(df.iloc[94][3])
            
            expected_cost_difference_our_policy_aa.append(float(df.iloc[95][3]))
            CI_cost_difference_our_cost_aa_cost.append(df.iloc[96][3])
            
            our_number_of_variables_constraints_LP.append(df.iloc[100][3])
            our_optimal_value_LP.append(float(df.iloc[101][3]))
            our_runtime_LP.append(float(df.iloc[102][3]))
            wm_number_of_variables_constraints_LP.append(df.iloc[103][3])
            wm_optimal_value_LP.append(float(df.iloc[104][3]))
            wm_runtime_LP.append(float(df.iloc[105][3]))
        
        elif nu == 0.01:
            expected_cost_our_policy.append(float(df.iloc[196][3]))
            expected_cost_aa.append(float(df.iloc[197][3]))
            expected_cost_will_ma.append(float(df.iloc[198][3]))
            
            expected_cost_difference_our_policy_wm.append(float(df.iloc[199][3]))
            CI_cost_difference_our_cost_wm_cost.append(df.iloc[200][3])
            
            expected_cost_difference_our_policy_aa.append(float(df.iloc[201][3]))
            CI_cost_difference_our_cost_aa_cost.append(df.iloc[202][3])
            
            our_number_of_variables_constraints_LP.append(df.iloc[206][3])
            our_optimal_value_LP.append(float(df.iloc[207][3]))
            our_runtime_LP.append(float(df.iloc[208][3]))
            wm_number_of_variables_constraints_LP.append(df.iloc[209][3])
            wm_optimal_value_LP.append(float(df.iloc[210][3]))
            wm_runtime_LP.append(float(df.iloc[211][3]))
            
        elif nu == 0.1:
            expected_cost_our_policy.append(float(df.iloc[302][3]))
            expected_cost_aa.append(float(df.iloc[303][3]))
            expected_cost_will_ma.append(float(df.iloc[304][3]))
            
            expected_cost_difference_our_policy_wm.append(float(df.iloc[305][3]))
            CI_cost_difference_our_cost_wm_cost.append(df.iloc[306][3])
            
            expected_cost_difference_our_policy_aa.append(float(df.iloc[307][3]))
            CI_cost_difference_our_cost_aa_cost.append(df.iloc[308][3])
            
            our_number_of_variables_constraints_LP.append(df.iloc[312][3])
            our_optimal_value_LP.append(float(df.iloc[313][3]))
            our_runtime_LP.append(float(df.iloc[314][3]))
            wm_number_of_variables_constraints_LP.append(df.iloc[315][3])
            wm_optimal_value_LP.append(float(df.iloc[316][3]))
            wm_runtime_LP.append(float(df.iloc[317][3]))
            
        elif nu == 0.2:
            expected_cost_our_policy.append(float(df.iloc[408][3]))
            expected_cost_aa.append(float(df.iloc[409][3]))
            expected_cost_will_ma.append(float(df.iloc[410][3]))
            
            expected_cost_difference_our_policy_wm.append(float(df.iloc[411][3]))
            CI_cost_difference_our_cost_wm_cost.append(df.iloc[412][3])
            
            expected_cost_difference_our_policy_aa.append(float(df.iloc[413][3]))
            CI_cost_difference_our_cost_aa_cost.append(df.iloc[414][3])
            
            our_number_of_variables_constraints_LP.append(df.iloc[418][3])
            our_optimal_value_LP.append(float(df.iloc[419][3]))
            our_runtime_LP.append(float(df.iloc[420][3]))
            wm_number_of_variables_constraints_LP.append(df.iloc[421][3])
            wm_optimal_value_LP.append(float(df.iloc[422][3]))
            wm_runtime_LP.append(float(df.iloc[423][3]))
        
        elif nu == 0.25:
            expected_cost_our_policy.append(float(df.iloc[514][3]))
            expected_cost_aa.append(float(df.iloc[515][3]))
            expected_cost_will_ma.append(float(df.iloc[516][3]))
            
            expected_cost_difference_our_policy_wm.append(float(df.iloc[517][3]))
            CI_cost_difference_our_cost_wm_cost.append(df.iloc[518][3])
            
            expected_cost_difference_our_policy_aa.append(float(df.iloc[519][3]))
            CI_cost_difference_our_cost_aa_cost.append(df.iloc[520][3])
            
            our_number_of_variables_constraints_LP.append(df.iloc[524][3])
            our_optimal_value_LP.append(float(df.iloc[525][3]))
            our_runtime_LP.append(float(df.iloc[526][3]))
            wm_number_of_variables_constraints_LP.append(df.iloc[527][3])
            wm_optimal_value_LP.append(float(df.iloc[528][3]))
            wm_runtime_LP.append(float(df.iloc[529][3]))
            
        elif nu == 0.3:
            expected_cost_our_policy.append(float(df.iloc[620][3]))
            expected_cost_aa.append(float(df.iloc[621][3]))
            expected_cost_will_ma.append(float(df.iloc[622][3]))
            
            expected_cost_difference_our_policy_wm.append(float(df.iloc[623][3]))
            CI_cost_difference_our_cost_wm_cost.append(df.iloc[624][3])
            
            expected_cost_difference_our_policy_aa.append(float(df.iloc[625][3]))
            CI_cost_difference_our_cost_aa_cost.append(df.iloc[626][3])
            
            our_number_of_variables_constraints_LP.append(df.iloc[630][3])
            our_optimal_value_LP.append(float(df.iloc[631][3]))
            our_runtime_LP.append(float(df.iloc[632][3]))
            wm_number_of_variables_constraints_LP.append(df.iloc[633][3])
            wm_optimal_value_LP.append(float(df.iloc[634][3]))
            wm_runtime_LP.append(float(df.iloc[635][3]))
            
        elif nu == 0.5:
            expected_cost_our_policy.append(float(df.iloc[726][3]))
            expected_cost_aa.append(float(df.iloc[727][3]))
            expected_cost_will_ma.append(float(df.iloc[728][3]))
            
            expected_cost_difference_our_policy_wm.append(float(df.iloc[729][3]))
            CI_cost_difference_our_cost_wm_cost.append(df.iloc[730][3])
            
            expected_cost_difference_our_policy_aa.append(float(df.iloc[731][3]))
            CI_cost_difference_our_cost_aa_cost.append(df.iloc[732][3])
            
            our_number_of_variables_constraints_LP.append(df.iloc[736][3])
            our_optimal_value_LP.append(float(df.iloc[737][3]))
            our_runtime_LP.append(float(df.iloc[738][3]))
            wm_number_of_variables_constraints_LP.append(df.iloc[739][3])
            wm_optimal_value_LP.append(float(df.iloc[740][3]))
            wm_runtime_LP.append(float(df.iloc[741][3]))
            
        elif nu == 0.7:
            expected_cost_our_policy.append(float(df.iloc[832][3]))
            expected_cost_aa.append(float(df.iloc[833][3]))
            expected_cost_will_ma.append(float(df.iloc[834][3]))
            
            expected_cost_difference_our_policy_wm.append(float(df.iloc[835][3]))
            CI_cost_difference_our_cost_wm_cost.append(df.iloc[836][3])
            
            expected_cost_difference_our_policy_aa.append(float(df.iloc[837][3]))
            CI_cost_difference_our_cost_aa_cost.append(df.iloc[838][3])
            
            our_number_of_variables_constraints_LP.append(df.iloc[842][3])
            our_optimal_value_LP.append(float(df.iloc[843][3]))
            our_runtime_LP.append(float(df.iloc[844][3]))
            wm_number_of_variables_constraints_LP.append(df.iloc[845][3])
            wm_optimal_value_LP.append(float(df.iloc[846][3]))
            wm_runtime_LP.append(float(df.iloc[847][3]))
        
        elif nu == 1:
            expected_cost_our_policy.append(float(df.iloc[938][3]))
            expected_cost_aa.append(float(df.iloc[939][3]))
            expected_cost_will_ma.append(float(df.iloc[940][3]))
            
            expected_cost_difference_our_policy_wm.append(float(df.iloc[941][3]))
            CI_cost_difference_our_cost_wm_cost.append(df.iloc[942][3])
            
            expected_cost_difference_our_policy_aa.append(float(df.iloc[943][3]))
            CI_cost_difference_our_cost_aa_cost.append(df.iloc[944][3])
            
            our_number_of_variables_constraints_LP.append(df.iloc[948][3])
            our_optimal_value_LP.append(float(df.iloc[949][3]))
            our_runtime_LP.append(float(df.iloc[950][3]))
            wm_number_of_variables_constraints_LP.append(df.iloc[951][3])
            wm_optimal_value_LP.append(float(df.iloc[952][3]))
            wm_runtime_LP.append(float(df.iloc[953][3]))
                
    results[nu]['expected_cost_our_policy'] = expected_cost_our_policy
    results[nu]['expected_cost_aa'] = expected_cost_aa
    results[nu]['expected_cost_will_ma'] = expected_cost_will_ma
    results[nu]['expected_cost_difference_our_policy_wm'] = expected_cost_difference_our_policy_wm
    results[nu]['CI_cost_difference_our_cost_wm_cost'] = CI_cost_difference_our_cost_wm_cost
    results[nu]['expected_cost_difference_our_policy_aa'] = expected_cost_difference_our_policy_aa
    results[nu]['CI_cost_difference_our_cost_aa_cost'] = CI_cost_difference_our_cost_aa_cost
    results[nu]['our_number_of_variables_constraints_LP'] = our_number_of_variables_constraints_LP
    results[nu]['our_optimal_value_LP'] = our_optimal_value_LP
    results[nu]['our_runtime_LP'] = our_runtime_LP
    results[nu]['wm_number_of_variables_constraints_LP'] = wm_number_of_variables_constraints_LP
    results[nu]['wm_optimal_value_LP'] = wm_optimal_value_LP
    results[nu]['wm_runtime_LP'] = wm_runtime_LP
    
    competitive_ratio[nu]['competitive_ratio'] = np.array(expected_cost_our_policy)/np.array(our_optimal_value_LP)

In [10]:
competitive_ratio

{0: {'competitive_ratio': array([1.23781688, 1.16477679, 1.17500506, 1.13012036, 1.04643971,
         1.23733448, 1.08205657, 1.26379657, 1.22264895, 0.95505276,
         1.15904545, 1.0867383 , 1.16118929, 1.32618236, 0.92028833,
         1.20727532, 1.15669762, 1.08054109, 0.83376692, 1.19656652,
         1.20409564, 0.96136525, 1.27714077, 1.19913124, 1.20001365,
         1.19576718, 1.23898557, 1.08912804, 0.984424  , 1.11120566,
         1.34698246, 1.23272747, 1.08317799, 1.20770592, 1.13462782,
         0.95804781, 0.99604969, 1.2412158 , 1.33920536, 1.25018289,
         1.19580404, 0.9796856 , 1.25173883, 1.18950602, 1.25957397,
         1.0396371 , 0.88354685, 1.25889201, 1.26401773, 1.2080939 ])},
 0.01: {'competitive_ratio': array([1.23781688, 1.16477679, 1.17500506, 1.13012036, 1.04643971,
         1.23733448, 1.08205657, 1.26379657, 1.22264895, 0.95505276,
         1.16396709, 1.0867383 , 1.16118929, 1.32618236, 0.92028833,
         1.20727532, 1.15669762, 1.08054109, 0.83

In [11]:
# data = competitive_ratio

# # Store the base array for comparison
# base_array = data[0]['competitive_ratio']

# # Initialize a dictionary to store the results
# comparison_results = {}

# # Iterate over each key in the dictionary except the first one (0)
# for key in list(data.keys())[1:]:  # Skip the first key by starting from the second element
#     # Compare arrays element-wise
#     comparison_results[key] = data[key]['competitive_ratio'] < base_array

# comparison_results

In [12]:
nu_values = [0, 0.01, 0.1, 0.2, 0.25, 0.3, 0.5, 0.7, 1]

for nu in nu_values:
    print(max(competitive_ratio[nu]['competitive_ratio']))

1.3469824595715334
1.3469824595715334
1.343745817698314
1.3956071650424227
1.4188055094972627
1.4440817157681198
1.6208814691594449
1.7600869130517671
2.0379582283938085


In [13]:
nu_values = [0, 0.01, 0.1, 0.2, 0.25, 0.3, 0.5, 0.7, 1]

for nu in nu_values:
    sorted_elements = sorted(competitive_ratio[nu]['competitive_ratio'], reverse=True)
    top_5_elements = sorted_elements[:5]
    print(top_5_elements)

[1.3469824595715334, 1.3392053578392467, 1.3261823642532595, 1.2771407699027402, 1.2640177315298098]
[1.3469824595715334, 1.3392053578392467, 1.3261823642532595, 1.2771407699027402, 1.2640177315298098]
[1.343745817698314, 1.3352832254036442, 1.3261823642532595, 1.3191629829137013, 1.3156785119034262]
[1.3956071650424227, 1.390177472496571, 1.3893824681107096, 1.3734225821912291, 1.3599198397202141]
[1.4188055094972627, 1.4052778355112736, 1.396126031087043, 1.3933743264210137, 1.390177472496571]
[1.4440817157681198, 1.4421042990815416, 1.433877520077933, 1.42034701917729, 1.4111829424317923]
[1.6208814691594449, 1.578953056538188, 1.5647181315692444, 1.5595210039632756, 1.5566718680940717]
[1.7600869130517671, 1.7515203391645318, 1.7229723571772038, 1.7181486446596328, 1.7061830727487401]
[2.0379582283938085, 1.98023209446246, 1.9585793166480678, 1.9442298330821448, 1.9348645118106433]
